refer: 

https://www.kaggle.com/code/kmldas/cifar10-resnet-90-accuracy-less-than-5-min

https://www.kaggle.com/code/ayushnitb/cifar10-custom-resnet-cnn-pytorch-97-acc

In [32]:
import os
import torchvision.transforms as tt
from torchvision.datasets import ImageFolder
import pandas as pd
import shutil
import random
from torch.utils.data import DataLoader


# Data Extraction and Reorganization

In [34]:
# manully download the dataset
data_dir = '../../dataset/CIFAR-10'
print(os.listdir(data_dir))

data_train_dir = data_dir + "/train"
data_test_dir = data_dir + "/test"
data_valid_dir = data_dir + "/valid"

# 创建 valid 文件夹，如果不存在
os.makedirs(data_valid_dir, exist_ok=True)

classes = os.listdir(data_train_dir)
print(classes)

['sampleSubmission.csv', 'test', 'train', 'trainLabels.csv', 'valid']
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [28]:
'''
# 读取trainLabels.csv文件
labels_csv_path = os.path.join(data_dir, 'trainLabels.csv')
labels_df = pd.read_csv(labels_csv_path)

# 确保标签文件夹存在
for label in labels_df['label'].unique():
    label_folder = os.path.join(data_train_dir, label)
    if not os.path.exists(label_folder):
        os.makedirs(label_folder)

# 移动图片到对应的标签文件夹中
for idx, row in labels_df.iterrows():
    img_id = row['id']
    label = row['label']
    
    # 源图片路径，假设图片后缀是 .png
    src_img_path = os.path.join(data_train_dir, f'{img_id}.png')
    
    # 目标图片路径
    dst_img_path = os.path.join(data_train_dir, label, f'{img_id}.png')
    
    # 移动图片到对应的标签文件夹
    if os.path.exists(src_img_path):
        shutil.move(src_img_path, dst_img_path)

print("图片移动完成！")

# 遍历 train 目录下的每个类别文件夹
for class_name in os.listdir(data_train_dir):
    class_train_dir = os.path.join(data_train_dir, class_name)
    
    # 确保是目录
    if os.path.isdir(class_train_dir):
        # 创建对应的 valid 类别文件夹
        class_valid_dir = os.path.join(data_valid_dir, class_name)
        os.makedirs(class_valid_dir, exist_ok=True)
        
        # 获取当前类别文件夹中的所有图片文件
        images = [img for img in os.listdir(class_train_dir) if img.endswith(('.png', '.jpg', '.jpeg'))]
        
        # 随机选择 20% 的图片进行移动
        num_images = len(images)
        num_valid = int(0.2 * num_images)
        valid_images = random.sample(images, num_valid)
        
        # 将选定的图片移动到 valid 类别文件夹
        for img in valid_images:
            src_img_path = os.path.join(class_train_dir, img)
            dst_img_path = os.path.join(class_valid_dir, img)
            shutil.move(src_img_path, dst_img_path)
        
        print(f"Moved {num_valid} images from {class_name} to valid/{class_name}")
'''

'\n# 读取trainLabels.csv文件\nlabels_csv_path = os.path.join(data_dir, \'trainLabels.csv\')\nlabels_df = pd.read_csv(labels_csv_path)\n\n# 确保标签文件夹存在\nfor label in labels_df[\'label\'].unique():\n    label_folder = os.path.join(data_train_dir, label)\n    if not os.path.exists(label_folder):\n        os.makedirs(label_folder)\n\n# 移动图片到对应的标签文件夹中\nfor idx, row in labels_df.iterrows():\n    img_id = row[\'id\']\n    label = row[\'label\']\n    \n    # 源图片路径，假设图片后缀是 .png\n    src_img_path = os.path.join(data_train_dir, f\'{img_id}.png\')\n    \n    # 目标图片路径\n    dst_img_path = os.path.join(data_train_dir, label, f\'{img_id}.png\')\n    \n    # 移动图片到对应的标签文件夹\n    if os.path.exists(src_img_path):\n        shutil.move(src_img_path, dst_img_path)\n\nprint("图片移动完成！")\n\n# 遍历 train 目录下的每个类别文件夹\nfor class_name in os.listdir(data_train_dir):\n    class_train_dir = os.path.join(data_train_dir, class_name)\n    \n    # 确保是目录\n    if os.path.isdir(class_train_dir):\n        # 创建对应的 valid 类别文件夹\n        c

# Data Preparation

## Use test set for validation
我们将简单地使用测试集作为验证集，而不是从训练集中留出一小部分（例如10%）的数据进行验证。这只是为训练提供了更多的数据。一般来说，一旦你使用固定的验证集选择了最好的模型架构和超参数，在整个数据集上重新训练相同的模型是一个好主意，只是为了给它一个小小的性能提升。

## Channel-wise data normalization
通道数据归一化：我们将通过减去平均值并除以每个通道的标准偏差来归一化图像张量。因此，每个通道的数据均值为0，标准差为1。规范化数据可以防止来自任何一个通道的值在训练时不成比例地影响损失和梯度，只需具有比其他通道更高或更宽的值范围。

## Randomized data augmentations
随机数据增强：我们将在从训练数据集加载图像时应用随机选择的转换。具体来说，我们将每张图像填充4个像素，然后随机裁剪大小为32 x 32像素，然后以50%的概率水平翻转图像。由于每次加载特定图像时都会随机动态地应用变换，因此模型在每个训练历元中看到的图像略有不同，这使得它可以更好地进行泛化。

在CIFAR-10数据集中，数据预处理通常包括归一化和数据增强（Data Augmentation）两部分。你提供的代码中，通过`torchvision.transforms`（简称`tt`）实现了这些操作。下面是对代码的详细解释：

### 1. **归一化（Normalization）**
   - CIFAR-10 的图像是 32x32 大小的彩色图像，像素值范围在 `[0, 1]` 或 `[0, 255]` 之间。
   - 归一化是将图像的像素值通过减去均值并除以标准差，使其符合标准正态分布。
   
   ```python
   stats = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
   ```
   - 这里的 `stats` 变量包含了每个通道（RGB）的均值和标准差：
     - **均值**： `(0.4914, 0.4822, 0.4465)` 对应于 RGB 通道的均值。
     - **标准差**： `(0.2023, 0.1994, 0.2010)` 对应于 RGB 通道的标准差。
   - 这些值是通过对整个 CIFAR-10 数据集统计得到的，用于将图像像素值归一化。

### 2. **数据增强（Data Augmentation）**
   数据增强用于提高模型的泛化能力，通过随机变换图像来生成不同的训练样本。这对模型训练有帮助，尤其是在数据集较小时。

   ```python
   train_tfms = tt.Compose([
       tt.RandomCrop(32, padding=4, padding_mode='reflect'), 
       tt.RandomHorizontalFlip(), 
       tt.ToTensor(), 
       tt.Normalize(*stats, inplace=True)
   ])
   ```

   - **`tt.RandomCrop(32, padding=4, padding_mode='reflect')`**：随机裁剪图像为 32x32 的大小，并在每个方向上增加 4 像素的填充。`padding_mode='reflect'` 表示使用反射填充边缘区域。
   - **`tt.RandomHorizontalFlip()`**：随机水平翻转图像（以 50% 的概率）。这种水平翻转在很多场景中可以保留图像的主要特征，特别适用于物体不需要有固定方向的任务。
   - **`tt.ToTensor()`**：将图像从 PIL 格式（或 numpy 数组）转换为 PyTorch 张量，并将像素值归一化到 `[0, 1]`。
   - **`tt.Normalize(*stats, inplace=True)`**：使用之前定义的均值和标准差对图像进行归一化，使每个像素的值服从标准正态分布。这一步是在转换为张量之后进行的。

### 3. **验证集预处理**
   验证集的预处理一般比训练集简单，因为不需要数据增强。验证集只需进行归一化即可：

   ```python
   valid_tfms = tt.Compose([tt.ToTensor(), tt.Normalize(*stats)])
   ```
   - 这里没有使用数据增强，只是将图像转换为张量并归一化。

### 总结
- **训练集**：使用了数据增强（随机裁剪和水平翻转）和归一化。
- **验证集**：只进行了归一化处理。
   
这些预处理步骤帮助模型更好地学习 CIFAR-10 数据集的特征，并且通过数据增强，增加了数据的多样性，减少过拟合的可能性。

见experiments/Transforms.ipynb



In [4]:
# Data transform (normalization and data augmentation)
# CIFAR 数据集的均值
stats = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))

transforms_train = tt.Compose([tt.RandomCrop(32, padding=4, padding_mode="reflect"),
                               tt.RandomHorizontalFlip(),
                               tt.ToTensor(),
                               tt.Normalize(*stats, inplace=True)])

transforms_valid = tt.Compose([tt.Normalize(*stats),
                             tt.ToTensor()])

In [29]:
# PyTorch datasets

train_ds = ImageFolder(data_train_dir, transforms_train)
valid_ds = ImageFolder(data_valid_dir, transforms_valid)

In [30]:
batch_size = 400

In [33]:
# PyTorch data loaders
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=3, pin_memory=True)
valid_dl = DataLoader(valid_ds, batch_size*2, num_workers=3, pin_memory=True)

In [ ]:
# define a method to show images
def show_batch(dl, n=1):
    